## Optimização de hyper-parâmetros

A ideia é que esta função selecione um número X de diferentes redes.

Depois vamos criar uma função que para cada rede, vai fazer o treino e testar o score com os dados de validação. Essa função depois ordena consoante o melhor valor de AUC.

In [23]:
from sklearn.model_selection import ParameterSampler
import random as r
def selecaoHyperParametros(d,neuronios,reg):
    lista_parametros = list(ParameterSampler(d, n_iter=4, random_state=10))
    r.seed(10)
    for var in lista_parametros:
        var['topologia'] = r.choices(neuronios,k=var['nrCamadas'])
        aux = var['regularizer']
        if aux == 1:
            var['l1'] = r.choice(reg)
        elif aux == 2:
            var['l2'] = r.choice(reg)
        elif aux == 3:
            var['l1'] = r.choice(reg)
            var['l2'] = r.choice(reg)
        else:
            pass
    return lista_parametros

In [24]:
from keras import models,layers,regularizers
def criaRede(param,inputSize):
    model=models.Sequential()
    aux = param['regularizer']
    kernel_reg = None
    '''if aux == 1:
        kernel_reg = regularizers.l1(param['l1'])
    elif aux == 2:
        kernel_reg = regularizers.l2(param['l2'])
    elif aux == 3:
        kernel_reg = regularizers.l1_l2(l1=param['l1'],l2=param['l2'])
    else:
        pass '''
    
    model.add(layers.Dense(param['topologia'][0],activation=param['ativacao'],
                           kernel_regularizer=kernel_reg,input_shape=(inputSize,)))
    if param['dropout'] > 0:
        model.add(layers.Dropout(param['dropout']))
    for var in param['topologia'][1:]:
        model.add(layers.Dense(var,activation=param['ativacao'],kernel_regularizer=kernel_reg))
        if param['dropout'] > 0:
            model.add(layers.Dropout(param['dropout']))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer=param['optimizer'],
    loss='binary_crossentropy',
    metrics=['accuracy'])
    return model

In [25]:
from sklearn.metrics import roc_curve, auc
from keras.callbacks import EarlyStopping
def optimizacaoHyperParametros(d,neuronios,reg,trainX,trainY,valX,valY):
    params = selecaoHyperParametros(d,neuronios,reg)
    for param in params:
        print(param)
        rede = criaRede(param,trainX.shape[1])
        if param['early_stopping'] > 0:
            early = EarlyStopping(monitor='val_loss',patience=param['early_stopping'])
            history = rede.fit(trainX,
                                trainY,
                                epochs=param['epochs'],
                                batch_size=param['batch_size'],
                                validation_data=(valX,valY),
                                  callbacks=[early])
        else:
            history = rede.fit(trainX,
                                trainY,
                                epochs=param['epochs'],
                                batch_size=param['batch_size'],
                                validation_data=(valX,valY))
        pred = rede.predict(valX)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(valY, pred)
        score = auc(false_positive_rate, true_positive_rate)
        param['score'] = score
    return params

## Aqui estão os parametros que vamos optimizar

### Faltam ainda ver o dropout, early stopping e regularização

### Optimizer temos de ver os parâmetros existentes também:
- rmsprop
    - rho: float >= 0.
    - epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
    - decay: float >= 0. Learning rate decay over each update.
- SGD
    - momentum: float >= 0. Parameter that accelerates SGD in the relevant direction and dampens oscillations.
    - decay: float >= 0. Learning rate decay over each update.
    - nesterov: boolean. Whether to apply Nesterov momentum.
- Adam
    - beta_1: float, 0 < beta < 1. Generally close to 1.
    - beta_2: float, 0 < beta < 1. Generally close to 1.
    - epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
    - decay: float >= 0. Learning rate decay over each update.
    - amsgrad: boolean. Whether to apply the AMSGrad variant of this algorithm from the paper "On the Convergence of Adam and Beyond"
    
##### Todos têm learning rate!
##### Existem ainda mais optimizers mas não sei se vale a pena ver todos!

In [31]:
dicionario = {
    'nrCamadas':[1,2,3,4,5,6,7,8],
    'ativacao':['relu','exponential','tanh','sigmoid','linear'],
    'epochs':[4,8,10,20],
    'batch_size':[64,128,256,512],
    'optimizer':['rmsprop','adam','sgd'],
    'dropout':[0.0,0.1,0.2,0.3,0.4],
    'regularizer':[0],#1,2,3],
    'early_stopping':[0],#2,3,4,5]
}
neuronios = [2,3,4,5,8,10,16,32,64]
valores_l1 = [0.1,0.01,0.001] 

In [34]:
import pandas as pd
ms = None
y = None
for df in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=60000):
    if ms is None:
        ms = df.iloc[:,0:100]
        y = df['HasDetections']
    else:
        y = pd.concat([y, df['HasDetections']], ignore_index=True)
        ms = pd.concat([ms,df.iloc[:,0:100]], ignore_index=True)
    print('Feito')

Feito
Feito
Feito
Feito
Feito
Feito
Feito
Feito
Feito
Feito


In [35]:
trainX = ms[:500000]
valX = ms[500000:550000]
trainY = y[:500000]
valY = y[500000:550000]

In [36]:
res = optimizacaoHyperParametros(dicionario,neuronios,valores_l1,
                          trainX,trainY,
                          valX,valY)

{'regularizer': 0, 'optimizer': 'sgd', 'nrCamadas': 6, 'epochs': 8, 'early_stopping': 0, 'dropout': 0.3, 'batch_size': 256, 'ativacao': 'relu', 'topologia': [10, 5, 10, 3, 32, 32]}
Train on 500000 samples, validate on 50000 samples
Epoch 1/8
500000/500000 [==============================] - 7s 14us/step - loss: 0.6933 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4998
Epoch 2/8
500000/500000 [==============================] - 6s 12us/step - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5016
Epoch 3/8
500000/500000 [==============================] - 6s 12us/step - loss: 0.6932 - acc: 0.4988 - val_loss: 0.6931 - val_acc: 0.4997
Epoch 4/8
500000/500000 [==============================] - 6s 12us/step - loss: 0.6932 - acc: 0.4999 - val_loss: 0.6932 - val_acc: 0.4998
Epoch 5/8
500000/500000 [==============================] - 6s 12us/step - loss: 0.6932 - acc: 0.4995 - val_loss: 0.6931 - val_acc: 0.5031
Epoch 6/8
500000/500000 [==============================] - 6s 12us/step - loss

In [37]:
resultado = pd.DataFrame(res)
resultado.sort_values(by=['score'],ascending=False)

,ativacao,batch_size,dropout,early_stopping,epochs,nrCamadas,optimizer,regularizer,score,topologia
2,relu,256,0.4,0,4,1,rmsprop,0,0.550640,[32]
3,sigmoid,512,0.0,0,20,7,adam,0,0.548682,"[10, 5, 4, 16, 8, 16, 10]"
1,sigmoid,512,0.0,0,20,8,rmsprop,0,0.547942,"[10, 3, 8, 4, 4, 64, 64, 2]"
0,relu,256,0.3,0,8,6,sgd,0,0.502012,"[10, 5, 10, 3, 32, 32]"


In [9]:
parametros = selecaoHyperParametros(dicionario,neuronios)
for var in parametros:
    print()
    print(var)


{'optimizer': 'sgd', 'nrCamadas': 6, 'epochs': 8, 'batch_size': 128, 'ativacao': 'sigmoid', 'topologia': [10, 5, 10, 3, 32, 32]}

{'optimizer': 'rmsprop', 'nrCamadas': 8, 'epochs': 20, 'batch_size': 512, 'ativacao': 'tanh', 'topologia': [10, 3, 8, 4, 4, 64, 64, 2]}

{'optimizer': 'sgd', 'nrCamadas': 8, 'epochs': 8, 'batch_size': 128, 'ativacao': 'exponential', 'topologia': [32, 10, 5, 4, 16, 8, 16, 10]}

{'optimizer': 'rmsprop', 'nrCamadas': 1, 'epochs': 4, 'batch_size': 256, 'ativacao': 'sigmoid', 'topologia': [3]}
